In [19]:
import os
import json
import requests

In [20]:
%matplotlib inline
%config Completer.use_jedi = False

In [270]:
link = "https://rinkeby.rarible.com/token/0x509fd4cdaa29be7b1fad251d8ea0fca2ca91eb60:111767?tab=details"
numbers = link.split('/')[-1].split(':')
contract, token = numbers[0], numbers[1].split('?')[0]

In [275]:
contract, token

('0x509fd4cdaa29be7b1fad251d8ea0fca2ca91eb60', '111767')

In [253]:
data = {
    "debug": "NO",
    "private_ext": TEST_BACK_PRIVATE
}
r = requests.post("http://localhost:8001/deploy", json=data)

In [258]:
transfer_contract = json.loads(r.content.decode())["link"]
transfer_contract

'0xd4Ba7daA7F3A5DbE2453210C4D458959687C20a6'

In [260]:
# add permission to contract (and only to the one) call transfer (so permission from nft owner),
# PS. it is possible to combine with the following (but no win?)

In [276]:
data = {
    "nft_contract": contract,
    "owner_address": transfer_contract,
    "token": token,
    "debug": "NO",
    "private_ext": DEFAULT_TESTNET_SENDER_PRIVATE_KEY
}
r = requests.post("http://localhost:8001/permission", json=data)

In [277]:
# set up pasword hash stright into contarct
# so, the password may know only sender and receiver: no need to trust to our back-end
data = {
    "transfer_contract": transfer_contract,
    "password": "TEST_PASSWORD",
    "nft_contract": contract,
    "token": token,
    "private_ext": DEFAULT_TESTNET_SENDER_PRIVATE_KEY
}
r = requests.post("http://localhost:8001/setpassword", json=data)

In [278]:
# book a gift for a receiver.
# We do it centralized with our back-end, so backend validate that receiver is preferable that sender wanted to (e.g. with help of created password)
# We send money (per gas) to the validated receiver (to his public address)
data = {
    "transfer_contract": transfer_contract,
    "receiver": MY_ACC_ADDRESS,
    "nft_contract": contract,
    "token": token,
    "private_ext": TEST_BACK_PRIVATE
}
r = requests.post("http://localhost:8001/book", json=data)

In [283]:
# do the transfer to receiver address.
# It may be called local from receiver frontend coz we sent him money
data = {
    "transfer_contract": transfer_contract,
    "receiver": MY_ACC_ADDRESS,
    "sender": DEFAULT_TESTNET_SENDER_ADDRESS,
    "nft_contract": contract,
    "token": token,
    "password": "TEST_PASSWORD",
    "private_ext": MY_ACC_PRIVATE
}
r = requests.post("http://localhost:8001/dtransfer", json=data)